#### This file contains the sentiment analysis model that will classify events as ```success``` or ```failure```

In [ ]:
import time

## Sklearn Implementation

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error
from sklearn.feature_extraction.text import CountVectorizer

In [3]:
sentiment140_sklearn = pd.read_parquet('../data-processed/sentiment140_model_data.parquet')

In [4]:
sentiment140_sklearn.head()

,text,label
0,love health4uandpet u guy r best,1
1,im meet one besti tonight cant wait girl talk,1
2,darealsunisakim thank twitter add sunisa got m...,1
3,sick realli cheap hurt much eat real food plu ...,1
4,lovesbrooklyn2 effect everyon,1


#### Preparing word embeddings

In [5]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(sentiment140_sklearn['text'])
y = sentiment140_sklearn['label']

In [6]:
Xtr, Xts, ytr, yts = train_test_split(
    X,
    y,
    test_size=0.3, 
    random_state=42
)

In [8]:
start_time_sklearn = time.time()
model_sklearn = LogisticRegression(random_state=42, max_iter=100).fit(Xtr.astype(int), ytr)
time_sklearn = time.time() - start_time_sklearn

In [10]:
mean_squared_error(yts, model_sklearn.predict(Xts.astype(int)))

0.37333333333333335

## Pyspark Implementation

In [11]:
import pyspark
import pyspark.sql.functions as F
import pyspark.sql.types as T
import os
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.feature import CountVectorizer

In [12]:
conf = pyspark.SparkConf()
conf.set('spark.ui.proxyBase', '/user/' + os.environ['JUPYTERHUB_USER'] + '/proxy/4041')
conf.set('spark.driver.memory','8g')
conf.set('spark.ui.showConsoleProgress', False)
try:
    sc = pyspark.SparkContext(conf=conf)
    spark = pyspark.SQLContext.getOrCreate(sc)
except:
    print('Spark context already exists, continuing with', sc)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/12/06 14:29:33 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/12/06 14:29:37 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
/opt/conda/envs/bigdata/lib/python3.10/site-packages/pyspark/sql/context.py:158: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [13]:
sentiment140_pyspark = spark.read.parquet('../data-processed/sentiment140_model_data.parquet')

#### Preparing word embeddings

In [14]:
sentiment140_pyspark = sentiment140_pyspark.withColumn('text', F.split(F.col('text'), ' '))

In [15]:
cv = CountVectorizer(inputCol='text', outputCol='embeddings')

In [16]:
model = cv.fit(sentiment140_pyspark)

In [17]:
sentiment140_pyspark = model.transform(sentiment140_pyspark)

In [18]:
sentiment140_pyspark = sentiment140_pyspark.withColumn('label', F.col('label').cast(T.IntegerType()))

In [19]:
sentiment140_pyspark.show(10)

+--------------------+-----+--------------------+
|                text|label|          embeddings|
+--------------------+-----+--------------------+
|[love, health4uan...|    1|(3216,[5,34,125,1...|
|[im, meet, one, b...|    1|(3216,[0,9,20,46,...|
|[darealsunisakim,...|    1|(3216,[12,15,39,5...|
|[sick, realli, ch...|    1|(3216,[24,28,37,6...|
|[lovesbrooklyn2, ...|    1|(3216,[133,1149,1...|
|[productoffear, t...|    1|(3216,[12,24,37,5...|
|[rkeithhil, than,...|    1|(3216,[55,102,265...|
|[keepinupwkri, je...|    1|(3216,[5,6,7,31,5...|
|[tommcfli, ah, co...|    1|(3216,[39,123,230...|
|[e4voip, respond,...|    1|(3216,[111,360,52...|
+--------------------+-----+--------------------+
only showing top 10 rows



In [20]:
train, test = sentiment140_pyspark.randomSplit([0.7, 0.3], seed=42)

In [21]:
model_pyspark = LogisticRegression(maxIter=100, featuresCol='embeddings', labelCol='label')

In [29]:
start_time_pyspark = time.time()
model_fitted_pyspark = model_pyspark.fit(train)
time_pyspark = start_time_pyspark - time.time()

In [24]:
model_predicted_pyspark = model_fitted_pyspark.transform(test)

In [26]:
model_predicted_pyspark.withColumn('prediction', F.col('prediction').cast(T.IntegerType()))\
.select(['text', 'label', 'prediction'])\
.show(10)

+--------------------+-----+----------+
|                text|label|prediction|
+--------------------+-----+----------+
|[21, day, till, c...|    1|         0|
|[a5hleyf, im, spe...|    0|         0|
|[aaronrva, bathro...|    0|         0|
|[across, univers,...|    1|         0|
|[adriman, roflmao...|    1|         0|
|           [aghsnow]|    0|         0|
|[ahh, tedium, fix...|    1|         0|
|[albinla, think, ...|    1|         1|
|[alicayaba, cuuut...|    0|         1|
|[allanatrogu, tha...|    1|         1|
+--------------------+-----+----------+
only showing top 10 rows



In [27]:
evaluator = RegressionEvaluator(predictionCol='prediction', labelCol='label', metricName='mse')
mse_spark = evaluator.evaluate(model_predicted_pyspark)

In [28]:
mse_spark

0.421875